In [1]:
import numpy as np

### Conjugate Gradient Method ###

* solve $Qx = b$ for $x$
* Medium ground between method of steepest descent (1st order, gradient) and Newton's method (2nd order, uses Hessian)

#### Conditions ####

* $Q$ is symmetric positive-definite

#### Conjugation ####

* The set of nonzero vectors $\{d_1, d_2,..., d_k\}$ are conjugate (also Q-orthogonal) with respect to $Q$ if

$\begin{equation}
d_i^{T} Q d_j = 0 \forall i \ne j
\end{equation}$

* If the set of vectors are Q-orthogonal, they are also linearly independent

#### Optimization Problem ####

Goal: $\min_{x \in \mathbb{R}^n} \frac{1}{2} x^T Q x - b^T x$

the unique solution to this problem is also the unique solution to $Qx = b$

Let $x^{*}$ denote the solution. Let $\{d_0, d_2,..., d_{n-1}\}$ be $Q$-conjugate. They are therefore a basis of the space, so

$x^{*} = \alpha_{0} d_{0} + ... + \alpha_{n-1} d_{n-1}$

and

$d_{i}^T Q x^{*} = d_{i}^T Q(\alpha_{0} d_{0} + ... + \alpha_{n-1} d_{n-1}) = \alpha_{i} d_{i}^TQd_{i}^T$

and

$\alpha_{i} = \dfrac{d_{i}^T Q x^{*}}{d_{i}^TQd_{i}^T}$, so

$x^{*} = \sum_{i=0}^{n-1} \dfrac{d_{i}^T b}{d_{i}^TQd_{i}^T} d_i$

Showing that we don't need to matrix invert $Q$ to solve for $x$.

#### Conjugate Direction Theorem ####

Let $\{d_0, d_2,..., d_{n-1}\}$ be $Q$-conjugate and $x_{0}$ an arbitrary starting point.

The update rule is

$x_{k+1} = x_{k} + \alpha_{k} d_{k}$ where
$g_{k} = Qx_{k} - b$ (gradient), and
$a_{k} = - \dfrac{g_{k}^T d_{k}}{d_{k}^T Q d_{k}} = - \dfrac{(Qx_{k} - b)^T d_{k}}{d_{k}^T Q d_{k}}$

After $n$ steps, $x_{n} = x^{*}$

#### Conjugate Gradient Method ####

We have the update rule $a_{k} = - \dfrac{g_{k}^T d_{k}}{d_{k}^T Q d_{k}} = - \dfrac{(Qx_{k} - b)^T d_{k}}{d_{k}^T Q d_{k}}$, but how should we choose the vectors $d_0,...,d_{n-1}$?

They are chosen on-the-fly, at each step of the algorithm.

Let $x_{i} \in \mathbb{R}^n$ be arbitrary.

$d_{0} = -g_{0} = b - Q x_{0}$
$\alpha_{k} = - \dfrac{g_{k}^T d_{k}}{d_{k}^T Q d_{k}}$
$x_{k+1} = x_{k} + \alpha_{k}d_{k}$
$g_{k} = Qx_{k} - b$
$d_{k+1} = - g_{k+1} + \Beta_{k} d_{k}$
$\Beta_{k} = \dfrac{g_{k+1}^T Q d_{k}}{d_{k}^T Q d_{k}}$

What's $\Beta_{}$

#### Resources ####

1. http://www.cs.cmu.edu/~aarti/Class/10725_Fall17/Lecture_Slides/conjugate_direction_methods.pdf

In [2]:
tol = 1e-3

In [3]:
def conjugate_gradient(A,b,tol):
    x = np.random.rand(b.shape[0]) # unif in [0,1)
    print("x generated: ", x)
    r = b - A @x    # calculate residual
    print("r beginning: ", r)
    if np.linalg.norm(r) < tol:
        return x
    p = r
    print("p: ", p)
    k = 0
    
    while(True):
        print("k: ", k)
        alpha = np.dot(r,r) / float(p.T @ A @ p)
        print("r dot r: ", np.dot(r,r))
        print("p.T @ A @ p: ", p.T @ A @ p)
        print("alpha: ", alpha)
        print("r: ", r)
        print("p: ", p)
        x = x + alpha * p
        r_old = r
        r = r_old - alpha * A @ p
        if np.linalg.norm(r) < tol:
            return x
        Beta = r.T @ r / r_old.T @ r_old
        print("Beta: ", Beta)
        p = r + Beta * p
        k = k + 1
        if k > 10 * b.shape[0]:
            return x

    

In [4]:
# Debugging on this

A = np.asarray([[1,0],[0,1]]) # A is 2x2 identity
b = np.asarray([4,4])
x_sol = conjugate_gradient(A,b,tol)
# x should equal 4

# why are we getting an exploding residual?

x generated:  [0.96165531 0.80993114]
r beginning:  [3.03834469 3.19006886]
p:  [3.03834469 3.19006886]
k:  0
r dot r:  19.408077824037136
p.T @ A @ p:  19.408077824037136
alpha:  1.0
r:  [3.03834469 3.19006886]
p:  [3.03834469 3.19006886]


In [5]:
import numpy as np
matrixSize = 10 
B = np.random.rand(matrixSize, matrixSize)
A = np.dot(B, B.T)

x = np.random.rand(matrixSize)
b = A @ x

In [6]:
print("A: ", A)
print("x: ", x)
print("b: ", b)


A:  [[3.24561921 2.68489126 2.55593817 3.30156445 1.63857533 2.04556376
  2.94782301 2.0426906  2.97846156 1.91869324]
 [2.68489126 3.76089903 3.27075201 3.41494076 2.84655951 3.17781354
  3.11280923 2.88123616 3.57153652 2.32737121]
 [2.55593817 3.27075201 3.62134849 3.32335696 2.49631885 2.61977163
  2.98872785 2.5452979  3.39664332 2.29098582]
 [3.30156445 3.41494076 3.32335696 4.34674013 2.14162612 2.49483369
  3.65497812 2.64690061 3.68826199 1.83523456]
 [1.63857533 2.84655951 2.49631885 2.14162612 2.6226045  2.51872853
  2.18598574 2.03973217 2.86021768 2.13679896]
 [2.04556376 3.17781354 2.61977163 2.49483369 2.51872853 3.00217379
  2.30298308 2.42454494 3.03470329 1.99531441]
 [2.94782301 3.11280923 2.98872785 3.65497812 2.18598574 2.30298308
  3.88035658 2.15151925 3.62794591 1.92351241]
 [2.0426906  2.88123616 2.5452979  2.64690061 2.03973217 2.42454494
  2.15151925 2.68194399 2.85340907 1.69671485]
 [2.97846156 3.57153652 3.39664332 3.68826199 2.86021768 3.03470329
  3.6279

In [7]:
x_sol = conjugate_gradient(A,b,tol)

x generated:  [0.40244188 0.70143596 0.38697253 0.17051943 0.14324417 0.08188788
 0.08647982 0.5608094  0.70575414 0.1219012 ]
r beginning:  [2.47197805 2.69620001 2.33624212 2.91623718 1.63589235 2.24282396
 2.29104411 2.03569987 2.193887   1.28852518]
p:  [2.47197805 2.69620001 2.33624212 2.91623718 1.63589235 2.24282396
 2.29104411 2.03569987 2.193887   1.28852518]
k:  0
r dot r:  50.91543397595289
p.T @ A @ p:  1393.0644379545317
alpha:  0.036549231025316514
r:  [2.47197805 2.69620001 2.33624212 2.91623718 1.63589235 2.24282396
 2.29104411 2.03569987 2.193887   1.28852518]
p:  [2.47197805 2.69620001 2.33624212 2.91623718 1.63589235 2.24282396
 2.29104411 2.03569987 2.193887   1.28852518]
Beta:  8.458735524458787
k:  1
r dot r:  0.8458735524458788
p.T @ A @ p:  99284.14077029552
alpha:  8.519724760502262e-06
r:  [ 0.35034997  0.13802962 -0.06434349  0.3166227  -0.26214797  0.1480061
 -0.11279228  0.0586393  -0.56233049 -0.42033636]
p:  [21.26015849 22.94447243 19.69731074 24.9843017